In [4]:
import pandas as pd

# Cargar el CSV
archivo_csv = r"D:\Soy Henry\Proyecto Final\metadata_top10.csv"  # Asegurate de que el path sea correcto
df = pd.read_csv(archivo_csv)

# Convertir reseñas a texto en minúsculas
df["category"] = df["category"].astype(str).str.lower()

# Definir palabras clave relacionadas con comida
food_keywords = {"food", "restaurant", "eat", "menu", "drink", "dish", "delicious", "taco", "boba", "meal", "dining", "cuisine", "flavor"}

# Filtrar los lugares que contienen al menos una palabra clave en las reseñas
df = df[df["category"].apply(lambda x: any(word in x for word in food_keywords))]



In [7]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Cargar el CSV
archivo_csv = "D:/Soy Henry/Proyecto Final/metadata_top10.csv"  # Cambia esto si es necesario
df = pd.read_csv(archivo_csv)

# Filtrar lugares de comida usando palabras clave en la columna de reseñas
palabras_clave = {"food", "restaurant", "eat", "menu", "drink", "dish", "delicious", "taco", "boba", "meal", "dining", "cuisine", "flavor"}
df["is_food"] = df["category"].astype(str).apply(lambda x: any(word in x.lower() for word in palabras_clave))
df = df[df["is_food"]]

# Eliminar duplicados y valores nulos esenciales
df = df.drop_duplicates()
df = df.dropna(subset=["address", "avg_rating", "num_of_reviews"])

# Asegurar que num_of_reviews es numérico
df["num_of_reviews"] = pd.to_numeric(df["num_of_reviews"], errors="coerce")

# Agrupar por ciudad y seleccionar el Top 10 de restaurantes
top_10_ciudad = df.groupby("city", group_keys=False).apply(lambda x: x.nlargest(10, "num_of_reviews"))

# Inicializar geolocalizador
geolocator = Nominatim(user_agent="restaurant_locator")

# Función para obtener coordenadas
def obtener_coordenadas(direccion):
    try:
        ubicacion = geolocator.geocode(direccion, timeout=10)
        if ubicacion:
            return pd.Series([ubicacion.latitude, ubicacion.longitude])
    except GeocoderTimedOut:
        return pd.Series([None, None])
    return pd.Series([None, None])

# Aplicar la función a las direcciones
top_10_ciudad[["latitude", "longitude"]] = top_10_ciudad["address"].apply(obtener_coordenadas)

# Guardar el resultado en un nuevo CSV
ruta_salida_csv = "D:/Soy Henry/Proyecto Final/top10_ciudad.csv"
top_10_ciudad.to_csv(ruta_salida_csv, index=False, encoding="utf-8")

print("✅ CSV con Top 10 restaurantes por ciudad y coordenadas guardado.")


C:\Users\calde\AppData\Local\Temp\ipykernel_8620\2582738527.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_10_ciudad = df.groupby("city", group_keys=False).apply(lambda x: x.nlargest(10, "num_of_reviews"))


✅ CSV con Top 10 restaurantes por ciudad y coordenadas guardado.
